In [ ]:
# ------------------------------------------------------------------------
# Deformable DETR
# Copyright (c) 2020 SenseTime. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 [see LICENSE for details]
# ------------------------------------------------------------------------
# Modified from DETR (https://github.com/facebookresearch/detr)
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# ------------------------------------------------------------------------


import os
print(os.getcwd())

import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader
import datasets
import util.misc as utils
import datasets.samplers as samplers
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model

from main import get_args_parser, main

def inference(args):
    device = torch.device(args.device)

    model, _, _ = build_model(args)
    model.to(device)

    if args.resume:
        if args.resume.startswith('https'):
            checkpoint = torch.hub.load_state_dict_from_url(
                args.resume, map_location='cpu', check_hash=True)
        else:
            checkpoint = torch.load(args.resume, map_location='cpu')

        missing_keys, unexpected_keys = model.load_state_dict(checkpoint['model'], strict=False)
        unexpected_keys = [k for k in unexpected_keys if not (k.endswith('total_params') or k.endswith('total_ops'))]
        if len(missing_keys) > 0:
            print('Missing Keys: {}'.format(missing_keys))
        if len(unexpected_keys) > 0:
            print('Unexpected Keys: {}'.format(unexpected_keys))
    model.eval()

    # manually load image and preprocess for inference
    dataset = datasets.coco.build("val", args)

    img, target = dataset[0]
    
    def denormalize(tensor, mean, std):
        for t, m, s in zip(tensor, mean, std):
            t.mul_(s).add_(m)
        return tensor
    denormalized_img = denormalize(img.clone(), [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # MS-DETR/datasets/coco.py:make_coco_transforms

    import matplotlib.pyplot as plt
    plt.imshow(denormalized_img.permute(1, 2, 0))

    with torch.no_grad():
        outputs = model(img.unsqueeze(0).to(device))

    print(outputs)

    # dissect outputs
    out_logits, out_bbox = outputs['pred_logits'], outputs['pred_boxes']

### Running the model
coco_path=/workspace/coco2017
num_gpus=3

EXP_DIR=/workspace/ms-detr-checkpoints/ms_detr_pp_300

GPUS_PER_NODE=$num_gpus ./tools/run_dist_launch.sh $num_gpus python -u main.py \
   --output_dir $EXP_DIR \
   --with_box_refine \
   --two_stage \
   --dim_feedforward 2048 \
   --epochs 12 \
   --lr_drop 11 \
   --coco_path=$coco_path \
   --num_queries 300 \
   --dropout 0.0 \
   --mixed_selection \
   --look_forward_twice \
   --use_ms_detr \
   --use_aux_ffn \
   --topk_eval 100 \
   --resume $EXP_DIR/checkpoint.pth \
   --eval

In [ ]:
parser = argparse.ArgumentParser('Deformable DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args('')

args.output_dir = r'/workspace/ms-detr-checkpoints/ms_detr_300'
args.resume = r'/workspace/ms-detr-checkpoints/ms_detr_300/checkpoint.pth'
args.with_box_refine = True
args.two_stage = True
args.dim_feedforward = 2048
args.epochs = 12
args.lr_drop = 11
args.coco_path = r'/workspace/coco2017'
args.num_queries = 300
args.use_ms_detr = True
args.use_aux_ffn = True
args.topk_eval = 100
args.eval = True
args.inference = True

if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
if args.inference:
    inference(args)
else:
    main(args)

In [ ]:
parser = argparse.ArgumentParser('Deformable DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args('')

args.output_dir = r'/workspace/ms-detr-checkpoints/ms_detr_300'
args.resume = r'/workspace/ms-detr-checkpoints/ms_detr_300/checkpoint.pth'
args.with_box_refine = True
args.two_stage = True
args.dim_feedforward = 2048
args.epochs = 12
args.lr_drop = 11
args.coco_path = r'/workspace/coco'
args.num_queries = 300
args.use_ms_detr = True
args.use_aux_ffn = True
args.topk_eval = 100
args.backbone = 'resnet18'
args.eval = True

device = torch.device(args.device)

model, _, _ = build_model(args)
model.to(device)

In [ ]:
# --------------------------------------------------------------------------------------------------------------------------
# Deformable DETR
# Copyright (c) 2020 SenseTime. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 [see LICENSE for details]
# --------------------------------------------------------------------------------------------------------------------------
# Modified from https://github.com/pytorch/pytorch/blob/173f224570017b4b1a3a1a13d0bff280a54d9cd9/torch/distributed/launch.py
# --------------------------------------------------------------------------------------------------------------------------

import subprocess
import os


nproc_per_node = 3
nnodes = 1
node_rank = 0
training_script_command = r'''python -u main.py \
   --output_dir /workspace/ms-detr-checkpoints/ms_detr_300_resnet18 \
   --with_box_refine \
   --two_stage \
   --dim_feedforward 2048 \
   --epochs 12 \
   --lr_drop 11 \
   --coco_path=/workspace/coco2017 \
   --num_queries 300 \
   --use_ms_detr \
   --use_aux_ffn \
   --topk_eval 100 \
   --backbone resnet18
   '''
#    --resume /workspace/ms-detr-checkpoints/ms_detr_300/checkpoint.pth \
#    --eval

# world size in terms of number of processes
dist_world_size = nproc_per_node * nnodes


# set PyTorch distributed related environmental variables
current_env = os.environ.copy()
current_env["MASTER_ADDR"] = '127.0.0.1'
current_env["MASTER_PORT"] = '29501'
current_env["WORLD_SIZE"] = str(dist_world_size)

processes = []

for local_rank in range(0, nproc_per_node):
    # each process's rank
    dist_rank = nproc_per_node * node_rank + local_rank
    current_env["RANK"] = str(dist_rank)
    current_env["LOCAL_RANK"] = str(local_rank)

    cmd = [training_script_command]

    process = subprocess.Popen(cmd, shell=True, env=current_env)
    processes.append(process)

for process in processes:
    process.wait()
    if process.returncode != 0:
        raise subprocess.CalledProcessError(returncode=process.returncode,
                                            cmd=process.args)

In [ ]:
import math
import os
import sys
from typing import Iterable

import torch
import util.misc as utils
from datasets.coco_eval import CocoEvaluator
from datasets.panoptic_eval import PanopticEvaluator
from datasets.data_prefetcher import data_prefetcher

def train_one_epoch(model: torch.nn.Module, criterion: torch.nn.Module,
                    data_loader: Iterable, optimizer: torch.optim.Optimizer,
                    device: torch.device, epoch: int, max_norm: float = 0):
    model.train()
    criterion.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('class_error', utils.SmoothedValue(window_size=1, fmt='{value:.2f}'))
    metric_logger.add_meter('grad_norm', utils.SmoothedValue(window_size=1, fmt='{value:.2f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 50

    prefetcher = data_prefetcher(data_loader, device, prefetch=True)
    samples, targets = prefetcher.next()

    # for samples, targets in metric_logger.log_every(data_loader, print_freq, header):
    for _ in metric_logger.log_every(range(len(data_loader)), print_freq, header):
        outputs = model(samples)
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        loss_dict_reduced_unscaled = {f'{k}_unscaled': v
                                      for k, v in loss_dict_reduced.items()}
        loss_dict_reduced_scaled = {k: v * weight_dict[k]
                                    for k, v in loss_dict_reduced.items() if k in weight_dict}
        losses_reduced_scaled = sum(loss_dict_reduced_scaled.values())

        loss_value = losses_reduced_scaled.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        if max_norm > 0:
            grad_total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        else:
            grad_total_norm = utils.get_total_grad_norm(model.parameters(), max_norm)
        optimizer.step()

        metric_logger.update(loss=loss_value, **loss_dict_reduced_scaled, **loss_dict_reduced_unscaled)
        metric_logger.update(class_error=loss_dict_reduced['class_error'])
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(grad_norm=grad_total_norm)

        samples, targets = prefetcher.next()
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}